In [141]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import random

In [142]:
RND_STATE = 777

Поскольку в дальнейшем планируется сравнение различных типов моделей, нам необходимо разделить наши данные на три выборки:
1. Тренировочная
2. Валидационная
3. Контрольная (тестовая)

Данное разделение необходимо для того, чтобы избежать переобучения на контрольную выборку.

In [143]:
data = pd.read_csv('Data\credit_train_processed.csv', encoding='cp1251', index_col=0)

In [144]:
data.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,M,NaN,NaN,UMN,59998.00,10,1_6,NaN,GRD,КРАСНОДАРСКИЙ,30000.0,1.0,1.0,0
1,F,NaN,MAR,UMN,10889.00,6,1_1,NaN,NaN,МОСКВА,NaN,2.0,0.0,0
2,M,32.0,MAR,SPC,10728.00,12,1_1,NaN,NaN,САРАТОВСКАЯ,NaN,5.0,0.0,0
3,F,27.0,NaN,SPC,12009.09,12,1_1,NaN,NaN,ВОЛГОГРАДСКАЯ,NaN,2.0,0.0,0
4,M,45.0,NaN,SPC,NaN,10,1_1,0.421385,SCH,ЧЕЛЯБИНСКАЯ,NaN,1.0,0.0,0


Разделим наши данные в соотношении 7:2:1.

In [145]:
train = data.sample(frac=0.7, random_state=RND_STATE).copy()

valid = data.drop(train.index)\
             .sample(frac=0.66, random_state=RND_STATE).copy()

test = data.drop(train.index).drop(valid.index).copy()

In [146]:
datasets = [train, valid, test]
datasets_names = ["Тренировочный", "Валидационный", "Контрольный"]
for df in datasets:
    print(f'{df.shape[0] / data.shape[0]:.2f}')

0.70
0.20
0.10


# Импутация пропусков

Давайте посмотрим количество пропусков по каждой переменной в наших наборах данных.

In [147]:
omission_info = pd.DataFrame(dict(zip(datasets_names, [df.isnull().sum() for df in datasets])))

In [148]:
omission_info

,Тренировочный,Валидационный,Контрольный
gender,0,0,0
age,1,1,1
marital_status,2,1,0
job_position,0,0,0
credit_sum,1,0,1
credit_month,0,0,0
tariff_id,0,0,0
score_shk,5,1,1
education,5,0,0
living_region,144,29,19


Пропуски в переменных будем проводить различными способами в зависимости от типа рассматриваемой переменной (категориальной или количественной). 

В случае *категориальной* переменной заполнение будем проводить <ins>модой</ins> **тренировочного** набора.

В случае *количественной* переменной заполнение проводится в зависимости от типа ее распределения (симметричный, ассиметричный):
* симметричный - можно воспользоваться <ins>средним</ins> значением **тренировочного** набора. Например, для параметра `age`.
* ассиметричный - лучше пользоваться <ins>медианой</ins> **тренировочного** набора, чтобы избежать влияния "черных лебедей".

Также можно заполнить значением выходящим за диапазон непропущенных значений, например, большим отрицательным числом.

## Количественные переменные

In [149]:
print(f'Количественные переменные: {test.dtypes[test.dtypes != object].index.values}')

Количественные переменные: ['age' 'credit_sum' 'credit_month' 'score_shk' 'monthly_income'
 'credit_count' 'overdue_credit_count' 'open_account_flg']


In [150]:
# Заполняем пропуски в переменных 'age', 'credit_sum', 'score_shk', 'monthly_income'
# медианой тренировочного набора.
for col in ['age', 'credit_sum', 'score_shk', 'monthly_income']:
    for df in datasets:
        df[col].fillna(train[col].median(), inplace=True)

In [151]:
# Для переменных 'credit_count' и 'overdue_credit_count' заполним пропуски значением -1
for col in ['credit_count', 'overdue_credit_count']:
    for df in datasets:
        df[col].fillna(-1, inplace=True)

## Категориальные переменные

In [152]:
print(f'Категориальные переменные: {test.dtypes[test.dtypes == object].index.values}')

Категориальные переменные: ['gender' 'marital_status' 'job_position' 'tariff_id' 'education'
 'living_region']


In [153]:
need_to_fix_categorical = omission_info.loc[test.dtypes[test.dtypes == object].index]\
                                        .sum(axis=1) > 0

In [154]:
need_to_fix_categorical[need_to_fix_categorical]

marital_status    True
education         True
living_region     True
dtype: bool

In [155]:
for cat in ['marital_status', 'education']:
    for df in datasets:
        df[cat].fillna(train[cat].mode().values[0], inplace=True)

In [156]:
for df in datasets:
        df['living_region'].fillna('НЕ УКАЗАН', inplace=True)

In [157]:
omission_info_after = pd.DataFrame(dict(zip(datasets_names, [df.isnull().sum() for df in datasets])))
omission_info_after

,Тренировочный,Валидационный,Контрольный
gender,0,0,0
age,0,0,0
marital_status,0,0,0
job_position,0,0,0
credit_sum,0,0,0
credit_month,0,0,0
tariff_id,0,0,0
score_shk,0,0,0
education,0,0,0
living_region,0,0,0


# Конструирование новых признаков (Feature Engineering)

Начнем конструирование новых признаков, исходя из знаний предметной области.

In [158]:
print(f'Категориальные переменные: {test.dtypes[test.dtypes != object].index}')

Категориальные переменные: Index(['age', 'credit_sum', 'credit_month', 'score_shk', 'monthly_income',
       'credit_count', 'overdue_credit_count', 'open_account_flg'],
      dtype='object')


In [159]:
# Рассчитаем ежемесячный платеж по кредиту для всех датасетов
for df in datasets:
    df['month_payment'] = df['credit_sum'] / df['credit_month']
# Заменим бесконечные значения, которые могли образоваться при делении, на -1
    df['month_payment'].replace([np.inf, -np.inf], -1, inplace=True)

In [160]:
# Также важным параметром является какую долю зарплаты человек будет отдавать на погашение кредита
for df in datasets:
    df['payment_income_part'] = df['month_payment'] / df['monthly_income']
# Заменим бесконечные значения, которые могли образоваться при делении, на -1    
    df['payment_income_part'].replace([np.inf, -np.inf], -1, inplace=True)

Теперь создадим переменные - средние значения количественной переменной, взятые по уровню категориальной переменной. Например, колонка `score_is_gt_median_education` отображает превосходит ли внутренняя скоринговая оценка клиента медианное значение для людей с таким же образованием.

In [171]:
categorical_cols = ['living_region', 'age', 'gender',
                     'marital_status', 'job_position', 'education']
quantitative_cols = ['monthly_income', 'score_shk']

category_col_names = dict(zip(categorical_cols, ['region', 'age', 'gender',
                                                 'marital', 'job', 'education']))
quantitative_col_names = dict(zip(quantitative_cols, ['income', 'score']))

for category in categorical_cols:
    for indicator in quantitative_cols:
        grouped = train.groupby(category).agg(np.median)[indicator]
        median_col_name = category_col_names[category] + '_median_' +  quantitative_col_names[indicator]
        is_gt_median_col_name = quantitative_col_names[indicator] + '_is_gt_median_' + category_col_names[category]                          
        for df in datasets:
            df[median_col_name] = df[category].map(lambda x: grouped[x])                      
            df[is_gt_median_col_name] = np.where(df[indicator] >= df[median_col_name], 'YES', 'NO')